In [1]:
import torch
import torch.nn as nn
from util import fed_avg, lottery_fl_v2, lottery_fl_v3, average_weights, copy_model, create_model, train, evaluate, prune_fixed_amount, get_prune_summary, get_prune_params, calculate_metrics
import math
from client import Client
from server import Server
from datasource import get_data, get_data_noniid, get_data_noniid_m
import multiprocessing as mp
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings(action='ignore')

torch.cuda.set_device('cuda:3')
def build_args(arch='mlp',
              dataset='mnist',
              data_split='iid',
              num_clients=10,
              lr=0.001,
              batch_size=4,
              comm_rounds=10,
              frac=0.3,
              client_epoch=10,
              acc_thresh=0.5,
              prune_iterations=None,
              prune_percent=0.80,
              prune_step=0.2,
              prune_type=None,
              train_verbosity=False,
              test_verbosity=False,
              prune_verbosity=False,
              val_freq=0,
              avg_logic="standalone",
              log_folder="log"
              ):

        
    args = type('', (), {})()
    args.arch = arch
    args.dataset = dataset
    args.data_split = data_split
    args.num_clients = num_clients
    args.lr = lr
    args.batch_size = batch_size
    args.comm_rounds = comm_rounds
    args.frac = frac
    args.client_epoch = client_epoch
    args.acc_thresh = acc_thresh
    args.prune_iterations = prune_iterations
    args.prune_percent = prune_percent
    args.prune_step= prune_step
    args.prune_type = prune_type
    args.train_verbosity = train_verbosity
    args.test_verbosity = test_verbosity
    args.prune_verbosity = prune_verbosity
    args.val_freq = val_freq
    args.avg_logic = avg_logic
    args.log_folder = log_folder
    return args


def run_experiment(args, client_update, server_update):

    train_loaders, test_loaders, test_loader = get_data_noniid(args.num_clients, # remove m for cifar10
                                           args.dataset,
                                           n_class = 3, nsamples = 100,#20,
                                           mode=args.data_split,
                                           batch_size=args.batch_size)

    clients = []
    
    for i in range(args.num_clients):
        clients.append(Client(args,
                              train_loaders[i],
                              test_loaders[i],
                              client_id=i))

    server = Server(args, 
                    clients, 
                    test_loader=test_loader)
    #server.test_server_update()
    server.fserver_update()
    return server, clients

experiments = [
    # This experiment's setting is all default
    {
        'args': build_args(client_epoch=10,
                            comm_rounds=40,
                            frac=1,
                            prune_step=0,
                            acc_thresh=1,
                            batch_size=32,
                            
                            #avg_logic="fed_avg",
                            #avg_logic="lottery_fl_v3",
                            data_split='non-iid',
                            arch = 'cnn',
                            dataset= 'cifar10',#'cifar10',
                            num_clients=40),
        'client_update': None,
        'server_update': None,
    },
]

experiment = experiments[0]
server, clients = run_experiment(experiment['args'],
               experiment['client_update'],
               experiment['server_update'])

print("###########################################################")
print(f"server acc {server.accuracies}")
print("###########################################################")
for i, c in enumerate(clients):
    print(f"client #{i} accuracies {c.accuracies}")
    print(f"client #{i} losses {c.losses}")
    print(f"client #{i} prune_rates {c.prune_rates}")
    print("\n\n\n")

Files already downloaded and verified
Files already downloaded and verified
Creating model for client 0
Copying model for client 0
Done Copying model for client 0
Creating model for client 1
Copying model for client 1
Done Copying model for client 1
Creating model for client 2
Copying model for client 2
Done Copying model for client 2
Creating model for client 3
Copying model for client 3
Done Copying model for client 3
Creating model for client 4
Copying model for client 4
Done Copying model for client 4
Creating model for client 5
Copying model for client 5
Done Copying model for client 5
Creating model for client 6
Copying model for client 6
Done Copying model for client 6
Creating model for client 7
Copying model for client 7
Done Copying model for client 7
Creating model for client 8
Copying model for client 8
Done Copying model for client 8
Creating model for client 9
Copying model for client 9
Done Copying model for client 9
Creating model for client 10
Copying model for client 

100%|██████████| 10/10 [00:00<00:00, 78.61it/s]
1.618908855431439e-05
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #7 *****FF
0.0
Evaluating: 100%|██████████| 94/94 [00:01<00:00, 81.46it/s]
Acc. for classes {9, 4, 6} truck deer frog  :  0.0034352836879432626
100%|██████████| 10/10 [00:00<00:00, 67.48it/s]
0.0
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #8 *****FF
0.0
Evaluating: 100%|██████████| 94/94 [00:01<00:00, 79.06it/s]
Acc. for classes {8, 1, 4} ship car deer  :  0.14860372340425532
100%|██████████| 10/10 [00:00<00:00, 68.85it/s]
0.0
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #9 *****FF
0.0
Evaluating: 100%|██████████| 63/63 [00:00<00:00, 75.40it/s]
Acc. for classes {9, 2} truck bird  :  0.0
100%|██████████| 10/10 [00:00<00:00, 58.63it/s]
0.0
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #10 *****FF
0.0
Evaluating: 100%|██████████| 94/

0.0
Evaluating: 100%|██████████| 63/63 [00:00<00:00, 90.30it/s]
Acc. for classes {8, 5} ship dog  :  0.003968253968253968
100%|██████████| 10/10 [00:00<00:00, 75.26it/s]
0.0
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #19 *****FF
0.0
Evaluating: 100%|██████████| 94/94 [00:01<00:00, 80.45it/s]
Acc. for classes {0, 9, 5} airplane truck dog  :  0.13242464539007093
100%|██████████| 10/10 [00:00<00:00, 59.24it/s]
0.0
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #20 *****FF
0.0
Evaluating: 100%|██████████| 94/94 [00:01<00:00, 79.20it/s]
Acc. for classes {0, 1, 2} airplane car bird  :  0.3318927304964539
100%|██████████| 10/10 [00:00<00:00, 65.53it/s]
0.0
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #21 *****FF
0.0
Evaluating: 100%|██████████| 94/94 [00:01<00:00, 78.88it/s]
Acc. for classes {3, 5, 6} cat dog frog  :  0.28468528368794327
100%|██████████| 10/10 [00:00<00:00, 62.10it/s]
0

100%|██████████| 10/10 [00:00<00:00, 58.55it/s]
0.0
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #30 *****FF
0.0
Evaluating: 100%|██████████| 63/63 [00:00<00:00, 80.09it/s]
Acc. for classes {2, 3} bird cat  :  0.4965277777777778
100%|██████████| 10/10 [00:00<00:00, 80.07it/s]
0.0
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #31 *****FF
0.0
Evaluating: 100%|██████████| 63/63 [00:00<00:00, 80.97it/s]
Acc. for classes {8, 4} ship deer  :  0.0
100%|██████████| 10/10 [00:00<00:00, 62.03it/s]
0.0
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #32 *****FF
0.0
Evaluating: 100%|██████████| 63/63 [00:00<00:00, 79.84it/s]
Acc. for classes {0, 5} airplane dog  :  0.0
100%|██████████| 10/10 [00:00<00:00, 66.81it/s]
0.0
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #33 *****FF
0.0
Evaluating: 100%|██████████| 94/94 [00:01<00:00, 86.91it/s]
Acc. for classes {1, 

client #:  5 with  0.0
Evaluating: 100%|██████████| 63/63 [00:00<00:00, 77.98it/s]
Acc. for classes {2, 3} bird cat  :  0.6200396825396826
client #:  6 with  0.0
Evaluating: 100%|██████████| 94/94 [00:01<00:00, 82.32it/s]
Acc. for classes {9, 4, 6} truck deer frog  :  0.6328679078014184
client #:  7 with  0.0
Evaluating: 100%|██████████| 94/94 [00:01<00:00, 84.62it/s]
Acc. for classes {8, 1, 4} ship car deer  :  0.671875
client #:  8 with  0.0
Evaluating: 100%|██████████| 63/63 [00:00<00:00, 85.72it/s]
Acc. for classes {9, 2} truck bird  :  0.7782738095238095
client #:  9 with  0.0
Evaluating: 100%|██████████| 94/94 [00:01<00:00, 83.13it/s]
Acc. for classes {0, 3, 5} airplane cat dog  :  0.5933067375886525
client #:  10 with  0.0
Evaluating: 100%|██████████| 94/94 [00:01<00:00, 84.21it/s]
Acc. for classes {8, 1, 4} ship car deer  :  0.5855496453900709
client #:  11 with  0.0
Evaluating: 100%|██████████| 63/63 [00:00<00:00, 80.47it/s]
Acc. for classes {1, 5} car dog  :  0.75992063492063

...

100%|██████████| 10/10 [00:00<00:00, 78.04it/s]
0.0
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #29 *****FF
0.0
Evaluating: 100%|██████████| 94/94 [00:01<00:00, 81.57it/s]
Acc. for classes {0, 5, 7} airplane dog horse  :  0.6695478723404256
100%|██████████| 10/10 [00:00<00:00, 76.59it/s]
0.0
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #30 *****FF
0.0
Evaluating: 100%|██████████| 63/63 [00:00<00:00, 82.36it/s]
Acc. for classes {2, 3} bird cat  :  0.6512896825396826
100%|██████████| 10/10 [00:00<00:00, 72.44it/s]
0.0
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #31 *****FF
0.0
Evaluating: 100%|██████████| 63/63 [00:00<00:00, 82.77it/s]
Acc. for classes {8, 4} ship deer  :  0.8819444444444444
100%|██████████| 10/10 [00:00<00:00, 66.23it/s]
0.0
num_pruned 0, num_params 61770, cur_prune_rate 0.0, prune_step: 0
FF***** Client #32 *****FF
0.0
Evaluating: 100%|██████████| 63/63 [00:00<

Evaluating: 100%|██████████| 94/94 [00:01<00:00, 88.28it/s]
Acc. for classes {0, 1, 3} airplane car cat  :  0.726063829787234
client #:  0 with  0.0
Evaluating: 100%|██████████| 63/63 [00:00<00:00, 85.14it/s]
Acc. for classes {1, 4} car deer  :  0.8521825396825397
client #:  1 with  0.0
Evaluating: 100%|██████████| 94/94 [00:01<00:00, 74.14it/s]
Acc. for classes {8, 3, 6} ship cat frog  :  0.7237367021276596
client #:  2 with  0.0
Evaluating: 100%|██████████| 63/63 [00:00<00:00, 84.73it/s]
Acc. for classes {9, 7} truck horse  :  0.7802579365079365
client #:  3 with  0.0
Evaluating: 100%|██████████| 94/94 [00:01<00:00, 84.50it/s]
Acc. for classes {0, 5, 7} airplane dog horse  :  0.692043439716312
client #:  4 with  0.0
Evaluating: 100%|██████████| 94/94 [00:01<00:00, 74.99it/s]
Acc. for classes {9, 2, 6} truck bird frog  :  0.6893838652482269
client #:  5 with  0.0
Evaluating: 100%|██████████| 63/63 [00:00<00:00, 76.01it/s]
Acc. for classes {2, 3} bird cat  :  0.6661706349206349
client 

client #13 accuracies [[0.28541667 0.340625   0.38229167 0.46354167 0.53541667 0.54166667
  0.56458333 0.56979167 0.65       0.64270833]
 [0.65520833 0.628125   0.63645833 0.73020833 0.71458333 0.728125
  0.70625    0.73958333 0.7625     0.753125  ]
 [0.77604167 0.765625   0.81041667 0.76979167 0.81770833 0.821875
  0.78125    0.84479167 0.85208333 0.86458333]
 [0.90104167 0.89479167 0.89791667 0.909375   0.953125   0.88333333
  0.90416667 0.94479167 0.953125   0.959375  ]
 [0.978125   0.96875    0.94791667 0.97291667 0.965625   0.984375
  0.9875     0.96875    0.978125   0.984375  ]
 [0.9875     0.98125    0.959375   0.990625   1.         1.
  0.99375    0.984375   0.9875     0.9875    ]
 [0.9875     0.9875     0.98125    1.         0.996875   1.
  0.99375    0.99375    1.         0.996875  ]
 [1.         0.99375    1.         0.98125    0.990625   1.
  0.996875   0.9875     0.990625   0.984375  ]
 [0.990625   0.990625   0.98125    0.99375    0.975      0.996875
  1.         0.996875 

client #30 losses [[1.54075313e+00 6.36284828e-01 6.08402073e-01 6.43711686e-01
  6.25652075e-01 5.52490115e-01 6.24772072e-01 5.25471091e-01
  4.92578119e-01 4.91774231e-01]
 [5.20690620e-01 4.23466057e-01 4.10921514e-01 3.98280174e-01
  3.97528440e-01 3.90391082e-01 3.91364306e-01 3.29730660e-01
  3.94082338e-01 3.29355150e-01]
 [3.16554129e-01 2.98455477e-01 2.49277562e-01 2.43515849e-01
  1.88161135e-01 1.79770589e-01 1.85668498e-01 2.16778785e-01
  1.36815339e-01 1.46860600e-01]
 [1.38141856e-01 8.98410603e-02 1.10789202e-01 8.50069150e-02
  7.20466524e-02 9.99419093e-02 4.95006666e-02 3.81762646e-02
  1.16120443e-01 3.89070809e-02]
 [4.43703644e-02 1.84075590e-02 2.58817188e-02 7.42196571e-03
  6.30263016e-02 3.92143317e-02 3.10201254e-02 1.33139323e-02
  2.33615227e-02 2.73422357e-02]
 [4.97498959e-02 1.23072509e-03 3.60566527e-02 2.35480964e-02
  1.63775440e-02 6.40687486e-03 3.14766238e-03 7.40273902e-03
  1.17519096e-01 1.39266010e-02]
 [1.09605826e-01 2.17478834e-02 1.696164

client #34 losses [[2.02188015e+00 1.17679608e+00 1.09570539e+00 1.09925902e+00
  1.09999347e+00 1.06307876e+00 1.01701522e+00 1.04911780e+00
  9.85983968e-01 9.49840367e-01]
 [9.58907425e-01 8.55409801e-01 8.59159350e-01 8.09831440e-01
  7.39501357e-01 7.51411796e-01 6.99048162e-01 6.28601491e-01
  5.50855279e-01 5.63207746e-01]
 [5.18421531e-01 4.82540935e-01 4.40647185e-01 3.68688941e-01
  3.42754990e-01 2.84523904e-01 2.31513351e-01 2.19537139e-01
  2.21693426e-01 2.10193366e-01]
 [1.81871086e-01 1.24145567e-01 1.19528964e-01 1.05930910e-01
  9.10139531e-02 9.87385064e-02 3.29119936e-02 4.96766642e-02
  2.86533944e-02 5.85723408e-02]
 [4.08098325e-02 4.10015248e-02 2.41445787e-02 4.49965224e-02
  3.92021537e-02 1.57566369e-02 1.32067529e-02 4.22811415e-03
  1.45920841e-02 3.94565053e-02]
 [6.54062405e-02 1.37785003e-02 1.09847309e-02 2.19987310e-03
  3.50306323e-03 4.49570315e-03 1.23918923e-02 1.86544308e-03
  1.99276023e-02 2.50184978e-03]
 [5.36631141e-03 1.04383263e-03 7.801518

In [ ]:
server.accuracies

In [ ]:
for c in clients:
    plt.plot(c.accuracies.flatten())

In [ ]:
import numpy as np

In [ ]:
commcost = np.zeros(40)
for i, c in enumerate(clients):
    for j,k in enumerate(c.prune_rates):
        if server.selected_client_tally[j][i] != 0.0:
            commcost[j]+=(1-k)

In [ ]:
commcost

In [2]:
server.client_accuracies.mean(axis=0)

array([0.68361393, 0.71223074, 0.72220164, 0.71264934, 0.7155502 ,
       0.7101114 , 0.71266279, 0.71570798, 0.71281939, 0.7149043 ,
       0.71408704, 0.71939326, 0.71969101, 0.72144506, 0.72083531,
       0.72170073, 0.72291561, 0.72293751, 0.72277498, 0.72291205,
       0.72281495, 0.72315347, 0.72320743, 0.72308632, 0.72301152,
       0.72318526, 0.72337444, 0.72318948, 0.72326006, 0.72314674,
       0.7230073 , 0.7232474 , 0.72315452, 0.72311099, 0.72304714,
       0.72303078, 0.72312695, 0.7230387 , 0.72314067, 0.72308447])

In [ ]:
server.selected_client_tally

In [ ]:
plt.plot(server.client_accuracies.mean(axis=0), label="lotfl")
plt.plot(acc2, label = "masked")
plt.legend()
plt.title("Mean Clients Acc. per Round")

In [ ]:
server.good_client_tally

In [ ]:
#standalone
server.client_accuracies.mean(axis=0)

In [ ]:
server.client_accuracies.mean(axis=0)

In [ ]:
server.client_accuracies

In [ ]:
max(server.client_accuracies.mean(axis=0))

In [ ]:
max(1, max(server.client_accuracies.mean(axis=0)))

In [ ]:
import torchvision as tv
import torchvision.transforms as tf
transform2 = tf.Compose([
            tf.ToTensor(),
            tf.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])
test_data = tv.datasets.CIFAR10(root="./data",
                        train=False,
                        download=True,
                        transform=transform2)
data_size_test = test_data.data.shape[0]

label_idx_pairs_test = [ (i, test_data.targets[i]) for i in range(data_size_test)]
label_idx_pairs_test = sorted(label_idx_pairs_test, key=lambda pair : pair[1])
sorted_idx_test = [idx for idx, label in label_idx_pairs_test]

test_digit_list = [[] for i in range(10)]

for i in range(len(sorted_idx_test)):
    test_digit_list[test_data.targets[i]].append(i)

test_loaders = []

for i in range(10):
    cur_sampler_test = torch.utils.data.BatchSampler(torch.utils.data.SubsetRandomSampler(test_digit_list[i]), 
                                                        32, 
                                                        drop_last=False)
    cur_loader_test = torch.utils.data.DataLoader(test_data,
                                                     batch_sampler=cur_sampler_test)

    test_loaders.append(cur_loader_test)

In [ ]:
from util import fevaluate
accs = [[] for i in range(10)]

for cnum, c in enumerate(clients):
    print("\n\n____________CLIENT ", cnum, "_________________\n\n")
    for i in range(10):
        eval_score = fevaluate(c.model, 
                                 test_loaders[i],
                                 verbose=True)
        if eval_score['Accuracy'][0] > 0.2 :
            accs[i].append(eval_score['Accuracy'])

In [ ]:
accs

In [ ]:
sum = 0
for i in range(10):
    if torch.tensor(accs[i]).mean(axis=0)[0]:
        sum += torch.tensor(accs[i]).mean(axis=0)[0]
    print(torch.tensor(accs[i]).mean(axis=0))
#(0.875+.946+.595+.725+.86+.76)/6
sum/10

In [ ]:
import numpy as np

In [ ]:
inv_prn_rates = []
inv_prn_tally = [[] for i in range(clients[0].args.comm_rounds)]
for cr in range(clients[0].args.comm_rounds):
    tally = server.selected_client_tally[cr]
    print(cr, tally)
    inv_prn_rates = []
    for i, c in enumerate(clients):
        inv_prn_rates.append(1-c.prune_rates[cr])
    inv_prn_tally[cr] = inv_prn_rates*tally

In [ ]:
commtot=np.sum(inv_prn_tally)

In [ ]:
commtot*8*61770/1024/1024